In [1]:
from contrans import contrans
import numpy as np
import pandas as pd
import dotenv
import os
import json
import requests 
import psycopg
import sqlite3
from sqlalchemy import create_engine    
dotenv.load_dotenv()
congresskey = os.getenv('congresskey')
postgrespassword = os.getenv('POSTGRES_PASSWORD')

In [2]:
ct = contrans()

In [3]:
#Creates a new, empty contrans database
dbserver, engine = ct.connect_to_postgres(ct.POSTGRES_PASSWORD, create_contrans=True)

In [4]:
members = ct.get_bioguideIDs() # members from Congress API
members = ct.make_cand_table(members) # joining the contributions ID with the Congress API data
terms, members = ct.terms_df(members) # separates the terms (non-atomic) data from members
ideology = ct.get_ideology() # gets the Ideology data from voteview.com
ct.make_members_df(members, ideology, engine) # joins members and ideology and uploads to postgres DB

In [5]:
ct.make_terms_df(terms, engine)

In [6]:
votes = ct.get_votes()
ct.make_votes_df(votes, engine)

In [7]:
# SQL queries
myquery = '''
SELECT *
FROM members
WHERE state = 'VA'
'''
pd.read_sql_query(myquery, con=engine)

,bioguideid,name,partyname,state,updatedate,url,depiction_attribution,depiction_imageurl,district,partyletter,...,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2
0,W000805,"Warner, Mark R.",Democratic,VA,2024-11-04T13:42:33Z,https://api.congress.gov/v3/member/W000805?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/w000805_20...,S,D,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
1,K000384,"Kaine, Tim",Democratic,VA,2024-11-04T13:42:21Z,https://api.congress.gov/v3/member/K000384?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/k000384_20...,S,D,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
2,K000399,"Kiggans, Jennifer A.",Republican,VA,2024-08-05T15:15:39Z,https://api.congress.gov/v3/member/K000399?for...,Image courtesy of the Member,https://www.congress.gov/img/member/66b0ce45b0...,02,R,...,None,0.258,0.431,-104.64907,0.90471,1045.0,34.0,None,0.258,0.430
3,G000595,"Good, Bob",Republican,VA,2024-06-08T18:40:22Z,https://api.congress.gov/v3/member/G000595?for...,Image courtesy of the Member,https://www.congress.gov/img/member/g000595_20...,05,R,...,None,0.800,-0.600,-77.56145,0.92833,1043.0,34.0,None,0.769,-0.639
4,W000825,"Wexton, Jennifer",Democratic,VA,2024-06-08T18:40:22Z,https://api.congress.gov/v3/member/W000825?for...,Image courtesy of the Member,https://www.congress.gov/img/member/w000825_20...,10,D,...,None,-0.384,0.302,-42.16114,0.95458,907.0,14.0,None,-0.427,0.174
5,S001209,"Spanberger, Abigail Davis",Democratic,VA,2024-06-08T18:40:22Z,https://api.congress.gov/v3/member/S001209?for...,Image courtesy of the Member,https://www.congress.gov/img/member/s001209_20...,07,D,...,None,-0.197,0.313,-105.49853,0.90510,1058.0,35.0,None,-0.262,0.291
6,C001118,"Cline, Ben",Republican,VA,2024-06-08T18:40:22Z,https://api.congress.gov/v3/member/C001118?for...,Image courtesy of the Member,https://www.congress.gov/img/member/c001118_20...,06,R,...,None,0.715,-0.212,-79.77979,0.92783,1065.0,30.0,None,0.709,-0.259
7,B001292,"Beyer, Donald S.",Democratic,VA,2024-06-08T18:40:22Z,https://api.congress.gov/v3/member/B001292?for...,Image courtesy of the Member,https://www.congress.gov/img/member/b001292_20...,08,D,...,None,-0.390,-0.100,-74.64359,0.93100,1044.0,31.0,None,-0.403,-0.216
8,G000568,"Griffith, H. Morgan",Republican,VA,2024-06-08T18:40:22Z,https://api.congress.gov/v3/member/G000568?for...,Image courtesy of the Member,https://www.congress.gov/img/member/g000568_20...,09,R,...,None,0.514,-0.350,-242.40046,0.78243,988.0,109.0,None,0.445,-0.278
9,C001078,"Connolly, Gerald E.",Democratic,VA,2024-06-08T18:40:22Z,https://api.congress.gov/v3/member/C001078?for...,Image courtesy of the Member,https://www.congress.gov/img/member/c001078_20...,11,D,...,None,-0.308,-0.045,-78.78733,0.92738,1045.0,32.0,None,-0.348,-0.208


In [8]:
myquery = '''
SELECT *
FROM votes
'''
data = pd.read_sql_query(myquery, con=engine)


In [33]:
# Create voting affinity table
myquery = '''
SELECT 
    a.icpsr AS icpsr1,
    b.icpsr AS icpsr2,
    AVG(CAST((a.cast_code = b.cast_code) AS INT)) AS agree
    FROM votes a
INNER JOIN votes b
    ON a.rollnumber = b.rollnumber 
    AND a.chamber = b.chamber
WHERE a.icpsr=14854 AND b.icpsr!=14854
GROUP BY icpsr1, icpsr2
ORDER BY agree DESC
'''
pd.read_sql_query(myquery, con=engine)

,icpsr1,icpsr2,agree
0,14854,21108,0.932651
1,14854,22378,0.927928
2,14854,29323,0.922421
3,14854,22124,0.913043
4,14854,22336,0.905371
...,...,...,...
443,14854,21566,0.271952
444,14854,21172,0.246914
445,14854,29573,0.230196
446,14854,22106,0.217391


In [42]:
myquery = '''
SELECT m.name, m.partyname, m.state, m.district, v.agree
FROM members m
INNER JOIN (
    SELECT 
        a.icpsr AS icpsr1,
        b.icpsr AS icpsr2,
        AVG(CAST((a.cast_code = b.cast_code) AS INT)) AS agree
        FROM votes a
    INNER JOIN votes b
        ON a.rollnumber = b.rollnumber 
        AND a.chamber = b.chamber
    WHERE a.icpsr=14854 AND b.icpsr!=14854
    GROUP BY icpsr1, icpsr2
    ORDER BY agree DESC
) v
    ON CAST(m.icpsr AS INT) = v.icpsr2
WHERE m.icpsr IS NOT NULL
ORDER BY v.agree DESC
'''
pd.read_sql_query(myquery, con=engine)

,name,partyname,state,district,agree
0,"Womack, Steve",Republican,AR,03,0.932651
1,"Fong, Vince",Republican,CA,20,0.927928
2,"Calvert, Ken",Republican,CA,41,0.922421
3,"Hinson, Ashley",Republican,IA,02,0.913043
4,"Kiley, Kevin",Republican,CA,03,0.905371
...,...,...,...,...,...
433,"Sablan, Gregorio Kilili Camacho",Democratic,MP,S,0.298507
434,"Bowman, Jamaal",Democratic,NY,16,0.286445
435,"Evans, Dwight",Democratic,PA,03,0.271952
436,"Bush, Cori",Democratic,MO,01,0.217391


In [ ]:
#indivs = pd.read_csv('data/CampaignFin22/indivs22.txt', quotechar="|", header=None,
#                     encoding = "ISO-8859-1")


/var/folders/4w/k9sqqcbx4dxgpjtwgv_1m29h0000gq/T/ipykernel_60044/819439748.py:1: DtypeWarning: Columns (6,13) have mixed types. Specify dtype option on import or set low_memory=False.
  indivs = pd.read_csv('data/CampaignFin22/indivs22.txt', quotechar="|", header=None,


In [ ]:
#indivs

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,2022,4061520221505841534,r0014256510,"DILLARD, DANIEL",C00000935,[24T Contribution],NaN,Z9500,08/29/2021,35,...,78557,DP,24T,C00401224,C00000935,M,2.022051e+17,NOT EMPLOYED,NOT EMPLOYED,Rept
1,2022,4061520221505841535,r0015503614,"WHITE, SCOTTO",C00633404,[24T Contribution],NaN,Z9500,08/29/2021,15,...,10701,PI,24T,C00401224,C00633404,M,2.022051e+17,NOT EMPLOYED,NOT EMPLOYED,Rept
2,2022,4061520221505841536,p0003861308,"DOMINGUEZ, SALLIE",C00632398,[24T Contribution],NaN,Z9500,08/29/2021,10,...,95011,PI,24T,C00401224,C00632398,F,2.022051e+17,HEALTHCARE,MCDONALD FAMILY DENTAL,Rept
3,2022,4061520221505841543,p0003689535,"BRADLEY, JANNETTE",C00678839,[24T Contribution],NaN,Z9500,08/29/2021,26,...,60453,PI,24T,C00401224,C00678839,F,2.022051e+17,LIBRARIAN,MEDICAL CENTER,Rept
4,2022,4061520221505841545,q0002181378,"BHATT, TANYA",C00580068,[24T Contribution],NaN,Z9500,08/29/2021,50,...,33141,PI,24T,C00401224,C00580068,F,2.022051e+17,MKTG,FRYE FINANCIAL,Rept
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66122633,2022,4021120221407214496,r0013324307,"STUART, CHARLES",C00000935,Accor Hotels,NaN,T9100,10/31/2021,15,...,92130,DP,15E,C00000935,NaN,M,2.022020e+17,HOTELIER,ACCOR HOTELS,Name
66122634,2022,4021120221407214497,p0004566720,"STUART, DAVID",C00000935,Food & Nutrient Impact LLC,NaN,Y4000,10/12/2021,50,...,17033,DP,15,C00000935,NaN,M,2.022020e+17,CONSULTANT,FOOD & NUTRIENT IMPACT LLC,
66122635,2022,4021120221407214498,q0001147584,"STUART, JANET",C00000935,General Motors,NaN,T2100,10/17/2021,25,...,49506,DP,15E,C00000935,NaN,F,2.022020e+17,INDIRECT MATERIAL BUYER,GENERAL MOTORS CORP.,PAC
66122636,2022,4021120221407214499,q0001147584,"STUART, JANET",C00000935,General Motors,NaN,T2100,10/25/2021,50,...,49506,DP,15E,C00000935,NaN,F,2.022020e+17,INDIRECT MATERIAL BUYER,GENERAL MOTORS CORP.,PAC
